In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1060


In [3]:
!pip install transformers

In [5]:
from transformers import pipeline

ModuleNotFoundError: No module named 'transformers'

In [6]:
print(pipeline('sentiment-analysis')('we love you'))

NameError: name 'pipeline' is not defined

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sent = "He remains characteristically confident and optimistic."
encoding = tokenizer.encode_plus(sent, add_special_tokens = True, truncation = True,
padding = "longest", return_attention_mask = True, return_tensors = "pt")

ModuleNotFoundError: No module named 'transformers'

In [8]:
encoding

NameError: name 'encoding' is not defined

In [9]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=73cbce8bb10cba83343b5d22abedb86efa6f001b25bb3fa3417e78308e9720f1
  Stored in directory: c:\users\gs63\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [10]:
import wget
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

ModuleNotFoundError: No module named 'wget'

In [11]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [12]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [13]:
path = '/content/drive/MyDrive/Studying/NLP/Group_Project/data/DREAM/train.json'

In [14]:
import pandas as pd
df_train = pd.read_json(path)

ValueError: Expected object or value

In [ ]:
df_train.head()


,0,1,2
0,[M: I am considering dropping my dancing class...,[{'question': 'What does the man suggest the w...,5-510
1,"[W: Well, I'm afraid my cooking isn't to your ...",[{'question': 'What does the man think of the ...,7-200
2,"[M: Look at the girl on the bike!, F: Oh, yes ...","[{'question': 'Where are the two persons?', 'c...",12-511
3,"[M: Can you accompany me while I am singing., ...",[{'question': 'What does the man want the woma...,16-83
4,[W: I can't figure out what's wrong with my wa...,"[{'question': 'What does the man mean?', 'choi...",5-231


In [ ]:
df_train.isnull().values.any()

False

In [ ]:
df_train[0][0]

['M: I am considering dropping my dancing class. I am not making any progress.',
 "W: If I were you, I stick with it. It's definitely worth time and effort."]

In [ ]:
df_train[1][0]

[{'answer': 'Continue her dancing class.',
  'choice': ['Consult her dancing teacher.',
   'Take a more interesting class.',
   'Continue her dancing class.'],
  'question': 'What does the man suggest the woman do?'}]

In [ ]:
df_train[2][0]

'5-510'

###preparing this data for fine-tuning

In [ ]:
#preparing this data for fine-tuning
# initialize lists for contexts, questions, choices and answers

def prepare_data(path):
  contexts = []
  questions = []
  choices_1 = []
  choices_2 = []
  choices_3 = []
  answers = []
  full_question = []


  df = pd.read_json(path)

  # contexts = df[0]
  for context in df[0]:
    contexts.append(context[0])
  for qa in df[1]:
    questions.append(qa[0]['question'])
    answer = qa[0]['answer']
    choices = qa[0]['choice']
    index = choices.index(answer)
    answers.append({'choice':index+1,'text':answer})
    
    choices_1.append(qa[0]['choice'][0])
    choices_2.append(qa[0]['choice'][1])
    choices_3.append(qa[0]['choice'][2])
    full_question.append('question:'+qa[0]['question']+'choices:'+'#/1/#'+qa[0]['choice'][0]+'&1&'+'#/2/#'+qa[0]['choice'][1]+'&2&'+'#/3/#'+qa[0]['choice'][2]+'&3&')
  

  return contexts, questions, choices_1,choices_2,choices_3, answers, full_question

train_contexts, train_questions, train_choices_1, train_choices_2, train_choices_3, train_answers, train_full_question = prepare_data(path)

print('train_contexts:',train_contexts[0],len(train_contexts))
print('train_questions:',train_questions[0])
print('train_choices_1:',train_choices_1[0])
print('train_choices_2:',train_choices_2[0])
print('train_choices_3:',train_choices_3[0])
print('train_answers:',train_answers[0],len(train_answers))
print('train_full_question:',train_full_question[0])



train_contexts: M: I am considering dropping my dancing class. I am not making any progress. 3869
train_questions: What does the man suggest the woman do?
train_choices_1: Consult her dancing teacher.
train_choices_2: Take a more interesting class.
train_choices_3: Continue her dancing class.
train_answers: {'choice': 3, 'text': 'Continue her dancing class.'} 3869
train_full_question: question:What does the man suggest the woman do?choices:#/1/#Consult her dancing teacher.&1&#/2/#Take a more interesting class.&2&#/3/#Continue her dancing class.&3&


In [ ]:
def add_end_idx(answers, questions):
    # loop through each answer-context pair
    for answer, questions in zip(answers, questions):
        # gold_text refers to the answer we are expecting to find in context
        gold_choice = answer['choice']
        gold_text = answer['text']
        answer_start = '#/'+str(gold_choice)+'/#'
        answer_end = '&'+str(gold_choice)+'&'
        # we already know the start index
        start_idx = questions.index(answer_start)+5
        # and ideally this would be the end index...
        end_idx = questions.index(answer_end)

        # ...however, sometimes squad answers are off by a character or two
        if questions[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_start'] = start_idx
            answer['answer_end'] = end_idx
        else:
            # this means the answer is off by 1-2 tokens
            for n in [1, 2]:
                if questions[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

            
# and apply the function to our two answer lists
add_end_idx(train_answers, train_full_question)w

In [ ]:
print('train_answers:',train_answers[0],len(train_answers))

train_answers: {'choice': 3, 'text': 'Continue her dancing class.', 'answer_start': 135, 'answer_end': 162} 3869


In [ ]:
# !git clone https://github.com/Microsoft/vcpkg.git\
# &&cd vcpkg\
# &&./bootstrap-vcpkg.sh\
# &&./vcpkg integrate install\
# &&./vcpkg install sentencepiece

In [ ]:
from transformers import DistilBertTokenizerFast
# initialize the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
# # tokenize

# from transformers import AlbertTokenizer, TFAlbertModel
# tokenizer = AlbertTokenizer.from_pretrained('ALBERT-xlarge-v2')
# model = TFAlbertModel.from_pretrained("ALBERT-xlarge-v2")

train_encodings = tokenizer(train_contexts, train_full_question, truncation=True, padding=True)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
##

##add feature

In [ ]:
# def add_token_feature(encodings, name,features):
#     new_features = []
#     # features = encodings(features, truncation=True, padding=True)
#     for i in range(len(features)):
#       new_features.append(encodings.char_to_token(i, features[i]))
#     print(len(new_features))
#     print(new_features[:10])
#     encodings.update({name: new_features})

# # apply function to our data
# add_token_feature(train_encodings, 'answers',train_answers)
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, train_answers)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

##use pytorch to built data object
\

In [ ]:
import torch


class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
train_dataset = SquadDataset(train_encodings)
test_dataset = SquadDataset(test_encodings)

In [ ]:
for key, val in train_encodings.items():
  print('key:',key[0],'val',val[0])

key: i val [101, 1049, 1024, 1045, 2572, 6195, 7510, 2026, 5613, 2465, 1012, 1045, 2572, 2025, 2437, 2151, 5082, 1012, 102, 3160, 1024, 2054, 2515, 1996, 2158, 6592, 1996, 2450, 2079, 1029, 9804, 1024, 1001, 1013, 1015, 1013, 1001, 23363, 2014, 5613, 3836, 1012, 1004, 1015, 1004, 1001, 1013, 1016, 1013, 1001, 2202, 1037, 2062, 5875, 2465, 1012, 1004, 1016, 1004, 1001, 1013, 1017, 1013, 1001, 3613, 2014, 5613, 2465, 1012, 1004, 1017, 1004, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
key: a val [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# import torch

# class SquadDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings,answers):
#         self.encodings = encodings
#         self.answers = answers

#     def __getitem__(self, item):
#       answers = str(self.answers[item])
#       encoding = self.encodings.encode_plus(
#       answers,
#       add_special_tokens=True,
#       max_length=self.max_len,
#       return_token_type_ids=False,
#        pad_to_max_length=True,
#       return_attention_mask=True,
#       return_tensors='pt',
#       )
#       return {'answers': review,'input_ids': encoding['input_ids'].flatten(),'attention_mask': encoding['attention_mask'].flatten()}

#     def __len__(self):
#         return len(self.encodings.input_ids)

# # build datasets for both our training and validation sets
# train_dataset = SquadDataset(train_encodings,train_answers)
# # val_dataset = SquadDataset(val_encodings)

In [ ]:
from transformers import DistilBertForQuestionAnswering
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
##Import model
# from transformers import AlbertModel
# model = AlbertModel.from_pretrained('albert-base-v2')
last_hidden_state, pooled_output = model(
  input_ids=encoding['input_ids'],
  attention_mask=encoding['attention_mask']
)

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(2):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 1: 100%|██████████| 242/242 [02:48<00:00,  1.44it/s, loss=1.02]


In [ ]:
# !git clone https://github.com/huggingface/transformers \
# && cd transformers \
# && git checkout a3085020ed0d81d4903c50967687192e3101e770 

In [ ]:
# !pip install ./transformers
# !pip install tensorboardX

In [ ]:
# !export SQUAD_DIR=/content/drive/MyDrive/Studying/NLP/Group_Project/data/DREAM \
# && python transformers/examples/run_squad.py \
#   --model_type albert \
#   --model_name_or_path albert-base-v2 \
#   --do_train \
#   --do_eval \
#   --do_lower_case \
#   --train_file $SQUAD_DIR/train.json \
#   --predict_file $SQUAD_DIR/dev.json \
#   --per_gpu_train_batch_size 12 \
#   --learning_rate 3e-5 \
#   --num_train_epochs 2.0 \
#   --max_seq_length 384 \
#   --doc_stride 128 \
#   --output_dir /content/model_output \
#   --save_steps 1000 \
#   --threads 4 \
#   --version_2_with_negative 